### 
*  [Update: NEW REFCAT ADDED](#REF)
    *  [Importing new astrometrical solutions](#REF3)
        *  [Creating new SLs with the new astrometrical solutions](#REF3.1) 
        *  [Creating new AL with the new SLs](#REF3.2) 

This is a notebook that runs in the AstroWISE environment. 

On the linux computer network of the Kapteyn Astronomical Institute execute:

linuxcommandline>module add awe

Make a local checkout of the AstroWISE code (in my case in /Users/users/verdoes/project/AstroWISE and checkout this branch:

linuxcommandline>cd /Users/users/verdoes/project/AstroWISE

linuxcommandline>git clone https://gitlab.astro-wise.org/omegacen/astro.git

linuxcommandline>git clone https://gitlab.astro-wise.org/omegacen/common.git

<span style="color:red">linuxcommandline>git checkout gvk/astromety-review</span>

linuxcommandline>setenv AWEPIPE Users/users/verdoes/project/AstroWISE

linuxcommandline>jawe notebook

The result should be that a jupyter notebook interactive user interface opens up in your browser. See this [AstroWISE howto](http://doc.astro-wise.org/man_howto_start.html) for details. 

Author: Gijs A. Verdoes Kleijn (g.a.verdoes.kleijn@rug.nl)

### Astrometric calibration using SCAMP inside AstroWISE, starting with as input a python list of SLID values and a astrometric reference catalog in csv format.
Took 4+19.40 hours with [make_with_scamp](https://gitlab.astro-wise.org/omegacen/astro/-/blob/gvk/astrometry-review/main/AstrometricParameters.py#L303).

In [1]:
!echo $AWEPIPE

/net/virgo01/data/users/mulder/MP/AstroWISE


#### BEGIN

In [4]:
#Input data and process configuration parameters
csv_filename_in='GaiaDR2.csv'
csv_file_in='./data/GaiaDR2.csv'
fits_filename_in='MasterCatalog_GaiaDR2.fits'
fits_file_in='./data/MasterCatalog_GaiaDR2.fits'
fits_filename_out=fits_filename_in[:-5]+'_readyToBecomeSourcelist.fits'
fits_file_out='./data/'+fits_filename_in[:-5]+'_readyToBecomeSourcelist.fits'
SLIDlist=[76154181,76154971,76154981,76154991,76155001,76155011,76155021,76155031,76155041,76155051,76155061,
76155071]

In [12]:
print(fits_filename_out)

MasterCatalog_GaiaDR2_readyToBecomeSourcelist.fits


Create a sourcelist from the mastercatalog in fits format, manually converted from the one in csv format using topcat, stealing from [IngestGAIA](https://github.com/willekemulder/masterproject/blob/gvk/gaia-based-astrom/notebooks/awe/IngestGAIA.ipynb) kindly provided by Ewout Helmich. Which in turns relies on functionality provided in [SourceCollections](https://gitlab.astro-wise.org/omegacen/astro/-/tree/master/main/sourcecollection), kindly provided by Hugo Buddelmeijer.

* FIXME: verify the ingestion as sourcelist can be omitted because the fits file ready to become a sourcelist already suffices to replace [set_refcat](https://gitlab.astro-wise.org/omegacen/astro/-/blob/gvk/astrometry-review/main/AstrometricParameters.py#L799)
* FIXME: remove the step of manually converting the csv format into fits: read csv directly and construct columns with pyfits.Column. 

In [4]:
import astro.config.startup



               Welcome to the Astro-WISE Environment

|01-Feb-19   Planned Groningen database downtime:
|             Tuesdays in 2019: January 8, February 5, March 5, April 2,
                                May 7, June 4, July 2, August 6, September 3,
                                October 8, November 5, December 3
|20-Nov-17   The Astro-WISE documentation has been converted from LaTex to
|            Sphinx. Documentation is now available at doc.astro-wise.org.

Importing Astro-WISE packages. Please wait...

Distributed Processing Unit: dpu.hpc.rug.astro-wise.org
Loaded 41 active dpu keys!
Dataserver: ds.astro.rug.astro-wise.org

Current profile: 
- username : AWWMULDER
- database : db.astro.rug.astro-wise.org
- project  : STUDENTWISE
- current privileges : 1 (MyDB)



In [1]:
from common.database.Context import context
context.set_project('OMEGACAM@VST', 3)

True

In [2]:
from astro.main.SourceList import SourceList
from astro.main.sourcecollection.External import External
#from astro.util.darma.common import new_table
import pyfits
import numpy
import time

In [3]:
hdus = pyfits.open(fits_file_in)

NameError: name 'fits_file_in' is not defined

In [22]:
nsources = len(hdus[1].data)
print(nsources)

230


In [23]:
# Columns A,B, POSANG and FLAG are mandatory columns for a SourceList (tbc by Ewout).
A = pyfits.Column(name='A', format='F6.4', array=numpy.ones(nsources))
B = pyfits.Column(name='B', format='F6.4', array=numpy.ones(nsources))
POSANG = pyfits.Column(name='POSANG', format='D', array=numpy.zeros(nsources))
FLAG = pyfits.Column(name='FLAG', format='D', array=numpy.zeros(nsources))

In [24]:
columns = hdus[1].data.columns + pyfits.ColDefs([A,B,POSANG,FLAG])

In [25]:
print(columns)

ColDefs(
    name = 'MASTER_ID'; format = 'I'
    name = 'MASTER_TYPE'; format = 'I'
    name = 'ORIGIN'; format = '8A'
    name = 'TIME_OBS'; format = '19A'; unit = 'iso-8601'
    name = 'TIME_ADD'; format = '19A'; unit = 'iso-8601'
    name = 'RA'; format = 'D'
    name = 'RA_ERR'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'DEC_ERR'; format = 'D'
    name = 'PM'; format = 'D'
    name = 'PM_ERR'; format = 'D'
    name = 'FLUX_ISO'; format = 'L'
    name = 'FLUXERR_ISO'; format = 'L'
    name = 'MAG_B1'; format = 'D'
    name = 'IS_VALID'; format = 'I'
    name = 'IS_CALIBRATOR'; format = 'I'
    name = 'IS_FASTMOVING'; format = 'L'
    name = 'IS_ADDITION'; format = 'L'
    name = 'A'; format = 'E'
    name = 'B'; format = 'E'
    name = 'POSANG'; format = 'D'
    name = 'FLAG'; format = 'D'
)


In [26]:
# To avoid burden of unnecessary implementation complexity I delete all columns which are not needed in the astrometric reference catalog.
columns.del_col('MASTER_ID')
columns.del_col('MASTER_TYPE')
columns.del_col('ORIGIN')
columns.del_col('TIME_OBS')
columns.del_col('TIME_ADD')
columns.del_col('PM')
columns.del_col('PM_ERR')
columns.del_col('FLUX_ISO')
columns.del_col('FLUXERR_ISO')
columns.del_col('IS_VALID')
columns.del_col('IS_CALIBRATOR')
columns.del_col('IS_FASTMOVING')
columns.del_col('IS_ADDITION')

ColDefs(
    name = 'RA'; format = 'D'
    name = 'RA_ERR'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'DEC_ERR'; format = 'D'
    name = 'MAG_B1'; format = 'D'
    name = 'A'; format = 'E'
    name = 'B'; format = 'E'
    name = 'POSANG'; format = 'D'
    name = 'FLAG'; format = 'D'
)

In [27]:
print(columns)

ColDefs(
    name = 'RA'; format = 'D'
    name = 'RA_ERR'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'DEC_ERR'; format = 'D'
    name = 'MAG_B1'; format = 'D'
    name = 'A'; format = 'E'
    name = 'B'; format = 'E'
    name = 'POSANG'; format = 'D'
    name = 'FLAG'; format = 'D'
)


In [28]:
now = time.time()

In [29]:
# create hdulist
hdulist = pyfits.HDUList()

# create primary hdu
prhdu = pyfits.PrimaryHDU()
prhdu.header.add_history('Created from scratch')
prhdu.header.add_history('Name = %s' % 'scratch')
prhdu.header.add_history('Date = %s' % time.ctime(now))
hdulist.append(prhdu)

# first OBJECTS tablehdu
#tbhdu = new_table(columns)
tbhdu = pyfits.BinTableHDU.from_columns(columns)
tbhdu.name = 'OBJECTS'
hdulist.append(tbhdu)

# fields tablehdu
fields_cols = [pyfits.Column(name='OBJECT_POS', format='1J', array=numpy.array([1])),
               pyfits.Column(name='OBJECT_COUNT', format='1J', array=numpy.array([nsources])),
               pyfits.Column(name='CHANNEL_NAME', format='16A', array=['OMEGACAM']),
               pyfits.Column(name='CHANNEL_NR', format='1J', array=numpy.array([1])),
               pyfits.Column(name='MAPNAXIS', format='1J', array=numpy.array([2])),
               pyfits.Column(name='DATE', format='32A', array=[time.strftime('%FT%H:%M:%S', time.gmtime(now))]),
             pyfits.Column(name='EPOCH', format='1D', array=numpy.array([2000.0]))]
#tbhdu = new_table(columns)
tbhdu = pyfits.BinTableHDU.from_columns(fields_cols)
tbhdu.name = 'FIELDS'
hdulist.append(tbhdu)

In [31]:
hdulist.writeto(fits_file_out, clobber=True)

In [33]:
fits_filename_in[:-5]

'MasterCatalog_GaiaDR2'

In [34]:
ext = External()
ext._SOURCELIST_DATA_NAME = fits_filename_in[:-5]
ext.name = fits_filename_out[:-5]
ext.make_from_fits(fits_file_out)

[fabricius.intra.astro.rug.nl] 2020-07-16T13:14:32 - Could not find row_identifier for fits file ./data/MasterCatalog_GaiaDR2_readyToBecomeSourcelist.fits, faking SeqNr


In [35]:
ext.commit()

In [36]:
ext.store_data()

[fabricius.intra.astro.rug.nl] 2020-07-16T13:14:37 - Creating sourcelist_data for External 4831881 .
[fabricius.intra.astro.rug.nl] 2020-07-16T13:14:47 - Created SourceList with SLID = 76551301, name = MasterCatalog_GaiaDR2 and 230 sources


In [37]:
sl_mastercatalog = (SourceList.SLID == 76551301)[0]
sl_mastercatalog.info()

SourceList: Name of SourceList : MasterCatalog_GaiaDR2
SourceList ID      : 76551301
Sources in list    : 230
Parameters in list : 13

 |
 +-COMBINE_METHOD: -1
 +-OBJECT: None
 +-SLID: 76551301
 +-associatelist: -1
 +-astrom_params: None
 +-chip: None
 +-creation_date: 2020-07-16 11:14:39.018099
 +-detection_frame: None
 +-filename: 
 +-filter: <astro.main.Filter.Filter object at 0x7f00bd834128>
 +-filters: 
 +-frame: None
 +-globalname: 
 +-instrument: None
 +-is_valid: 1
 +-llDEC: 0.753651345876656
 +-llRA: 325.223278152522
 +-lrDEC: 0.753651345876656
 +-lrRA: 325.098786682972
 +-name: MasterCatalog_GaiaDR2
 +-number_of_sources: 230
 +-object_id: 'AA8945BAC6826E3FE053174A17AC4B7D'
 +-process_params: <astro.main.SourceList.SourceListParameters object at 0x7f00bd7f3978>
 +-sexconf: <astro.main.Config.SextractorConfig object at 0x7f00bd7f3080>
 +-sexparam: <class 'common.database.typed_list.typed_list'>(<class 'str'>, [])
 +-sources: {'USER_47_f': <class 'float'>, 'USER_0_f': <class 'co

In [23]:
dir(ext)

['OBJECT',
 'PROCESS_TIME',
 'SCID',
 'STATUS_COMPARE',
 'STATUS_INSPECT',
 'STATUS_MAKE',
 'STATUS_VERIFY',
 '_IS_CAL',
 '_IS_CONFIG',
 '_IS_RAW',
 '_IS_SCIENCE',
 '_IS_SEQ',
 '_IS_SUPPORT',
 '_SCT',
 '_SOURCELIST_DATA_NAME',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cached_tableconverter_data',
 '_created',
 '_creator',
 '_delete',
 '_delete_post',
 '_desc',
 '_dirty',
 '_exist',
 '_get_onthefly',
 '_get_universe_sourcelist',
 '_inverses',
 '_modified',
 '_origin',
 '_privileges',
 '_project',
 '_publish',
 '_select_generic',
 '_tried_load_kids',
 '_updated',
 '_uptodate',
 '_validate_flags',
 '_version',
 'action',
 'add_sourcelist_data_to_relations',
 'after_init',
 

#### Astrometric calibration using above's astrometric reference catalog and using SCAMP:


In [44]:
results,justincase=[],[]
for SLID_in in SLIDlist:
    reduced=(SourceList.SLID==SLID_in)[0].frame
    astrom_params = AstrometricParameters()
    astrom_params.set_config(scamp=True)

    #Set instrument specific default config properties
    from astro.instrument.ConfigUpdaterFactory import create_config_updater
    instrument = reduced.instrument.name
    filter = reduced.filter.name
    chip = reduced.chip.name
    config_updater = create_config_updater(instrument)
    config_updater.update(astrom_params, filter=filter, chip=chip)

    astrom_params.reduced = reduced
    astrom_params.reduced.retrieve()
    astrom_params.reduced.weight.retrieve()
    astrom_params.process_params.REFCAT_SLID=76551301
    astrom_params.process_params.REFCAT='GAIA-DR2'
    
    astrom_params.make_with_scamp()
    astrom_params.verify()
    #astrom_params.inspect()
    astrom_params.commit()
    results.append((SLID_in,astrom_params.object_id))
    justincase.append((SLID_in,astrom_params,str(astrom_params.object_id)))
    # Inspect figures MUST be made after the ProcessTarget
    # is committed and an object_id assigned to it.
    astrom_params.make_inspect_figures()
    for fig in astrom_params.figures:
        fig.store()
        fig.commit()

[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:26 - Retrieving Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4509118-d4b707aaab715eea0e4f31d755a17cae52ea6fe6.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:26 - Retrieved Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4509118-d4b707aaab715eea0e4f31d755a17cae52ea6fe6.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:26 - Creating catalog on disk for SourceList MasterCatalog_GaiaDR2: tmp1594901066.286599.b10c00bf-7340-4433-87c1-32e004330789.refcat.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:26 - Copying RA -> Ra
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:26 - Copying DEC -> Dec
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:26 - No mag column found in SourceList MasterCatalog_GaiaDR2: ['SeqNr', 'B', 'SID', 'RA', 'A', 'USER_47_f', 'HTM', 'POSANG', 'FLAG', 'SLID', 'DEC', 'USER_0_f', 'USER_63_f']
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:26 - Creating tmp1594901066.286599

[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - The seeing (FWHM) is   0.95 arcsec
Found 193 associates out of 448 sources from 2 catalogs in 0.00 seconds      
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Clipping 2 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Clipping 5 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Clipping 3 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Clipping 2 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Clipping 1 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Clipping 1 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Clipped a total of 14 out of 193 point(s)
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - SETTING MEAN_DRA  to -0.0021636782662
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - SETTING MEAN_DDEC to 0.00888751835113
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - SETTING SIG_DRA   to 0.0

/net/virgo01/data/users/mulder/MP/AstroWISE/astro/main/AstrometricParameters.py:552: PyfitsDeprecationWarning: The new_table function is deprecated as of version 3.3 and may be removed in a future version.

        Use :meth:`BinTableHDU.from_columns` for new BINARY tables or :meth:`TableHDU.from_columns` for new ASCII tables instead.
  tbhdu = pyfits.new_table(cols)


[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Stored QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5030832-1d9c7a5e6aa6091fb31a5a57b748d615d729471a.fits[19kB] in 0.19 seconds (103.68kBps)
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Inspecting AstrometricParameters for Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4509118-d4b707aaab715eea0e4f31d755a17cae52ea6fe6.fits . . .
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Retrieving QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5030832-1d9c7a5e6aa6091fb31a5a57b748d615d729471a.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:29 - Retrieved QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5030832-1d9c7a5e6aa6091fb31a5a57b748d615d729471a.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:30 - Inspected Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4509118-d4b

[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - The seeing (FWHM) is   1.12 arcsec
Found 172 associates out of 414 sources from 2 catalogs in 0.00 seconds      
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - Clipping 2 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - Clipping 4 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - Clipping 3 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - Clipping 3 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - Clipping 1 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - Clipped a total of 13 out of 172 point(s)
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - SETTING MEAN_DRA  to 0.0194666960066
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - SETTING MEAN_DDEC to 0.0106855471189
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - SETTING SIG_DRA   to 0.0467477688177
[fabricius.intra.astro.rug.nl] 2020-07-16T14:04:33 - SETTING SIG_DDEC

...

/net/virgo01/data/users/mulder/MP/AstroWISE/astro/main/AstrometricParameters.py:552: PyfitsDeprecationWarning: The new_table function is deprecated as of version 3.3 and may be removed in a future version.

        Use :meth:`BinTableHDU.from_columns` for new BINARY tables or :meth:`TableHDU.from_columns` for new ASCII tables instead.
  tbhdu = pyfits.new_table(cols)


[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:11 - Stored QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5035657-d2f45c5f1a180d53ff952b14b612ee75e9091dbd.fits[19kB] in 0.26 seconds (75.77kBps)
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:11 - Inspecting AstrometricParameters for Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4806177-34c3c97e5d35c82f6ce0262c776e1d5c4ef864f5.fits . . .
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:11 - Retrieving QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5035657-d2f45c5f1a180d53ff952b14b612ee75e9091dbd.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:11 - Retrieved QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5035657-d2f45c5f1a180d53ff952b14b612ee75e9091dbd.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:11 - Inspected Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4806177-34c3

[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - The seeing (FWHM) is   0.80 arcsec
Found 194 associates out of 446 sources from 2 catalogs in 0.00 seconds      
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Clipping 1 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Clipping 4 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Clipping 3 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Clipping 1 point(s) . . . 
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Clipped a total of 9 out of 194 point(s)
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - SETTING MEAN_DRA  to 0.0384254738359
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - SETTING MEAN_DDEC to 0.00665153103962
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - SETTING SIG_DRA   to 0.0356508344913
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - SETTING SIG_DDEC  to 0.0247659850221
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - SETTIN

/net/virgo01/data/users/mulder/MP/AstroWISE/astro/main/AstrometricParameters.py:552: PyfitsDeprecationWarning: The new_table function is deprecated as of version 3.3 and may be removed in a future version.

        Use :meth:`BinTableHDU.from_columns` for new BINARY tables or :meth:`TableHDU.from_columns` for new ASCII tables instead.
  tbhdu = pyfits.new_table(cols)


[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Stored QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5036156-fdd3f4492f9aa3762ab14f8daf5b31918b384117.fits[22kB] in 0.19 seconds (120.02kBps)
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Inspecting AstrometricParameters for Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.6106413-ebcfe48156cdb23909d31fb9df7d26a814f9965b.fits . . .
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Retrieving QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5036156-fdd3f4492f9aa3762ab14f8daf5b31918b384117.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:15 - Retrieved QC-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-SCAMP-AstromParams---Residual-59046.5036156-fdd3f4492f9aa3762ab14f8daf5b31918b384117.fits
[fabricius.intra.astro.rug.nl] 2020-07-16T14:05:16 - Inspected Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.6106413-ebc

#### Results are here:

In [39]:
for result in results:
    print(result[0], result[1] ,AstrometricParameters(object_id=result[1]).creation_date)

76154181 'AA7C6381AF570A90E053174A17ACCCA9' 2020-07-16 11:45:18.708324
76154971 'AA7C6381AF5F0A90E053174A17ACCCA9' 2020-07-16 11:45:31.789889
76154981 'AA7C6381AF670A90E053174A17ACCCA9' 2020-07-16 11:45:39.491626
76154991 'AA7C6381AF6F0A90E053174A17ACCCA9' 2020-07-16 11:45:47.682608
76155001 'AA7C6381AF770A90E053174A17ACCCA9' 2020-07-16 11:46:05.818591
76155011 'AA7C6381AF7F0A90E053174A17ACCCA9' 2020-07-16 11:46:16.082616
76155021 'AA7C6381AF870A90E053174A17ACCCA9' 2020-07-16 11:46:24.608829
76155031 'AA7C6381AF8F0A90E053174A17ACCCA9' 2020-07-16 11:46:32.036720
76155041 'AA7C6381AF970A90E053174A17ACCCA9' 2020-07-16 11:46:55.553315
76155051 'AA7C6381AF9F0A90E053174A17ACCCA9' 2020-07-16 11:47:03.961748
76155061 'AA7C6381AFA70A90E053174A17ACCCA9' 2020-07-16 11:47:11.596655
76155071 'AA7C6381AFAF0A90E053174A17ACCCA9' 2020-07-16 11:47:21.858938


```
76154181 2020-07-15 11:07:15.672871
76154971 2020-07-15 11:07:27.800698
76154981 2020-07-15 11:07:41.184674
76154991 2020-07-15 11:07:53.397970
76155001 2020-07-15 11:08:06.126560
76155011 2020-07-15 11:08:18.752287
76155021 2020-07-15 11:08:31.397719
76155031 2020-07-15 11:08:41.424390
76155041 2020-07-15 11:08:52.734014
76155051 2020-07-15 11:09:02.831896
76155061 2020-07-15 11:09:15.505703
76155071 2020-07-15 11:09:29.177374
```

#### END

In [43]:
for slid in SLIDlist:
    redsframe = (SourceList.SLID == slid)[0].frame
    redsfile = (SourceList.SLID == slid)[0].frame.filename
    creationdate = (AstrometricParameters.reduced == redsframe).order_by('creation_date')[-1].creation_date
    for p in (AstrometricParameters.reduced == redsframe).order_by('creation_date'):
        print(p.process_params.REFCAT, p.CD1_1, '        ', p.creation_date)
    break


2MASS-PSC 5.931136941115944e-05          2020-05-14 20:41:44.539956
2MASS-PSC 5.931136941115944e-05          2020-05-15 07:57:37.977493
USNO-B1.0 5.9481800163997664e-05          2020-05-17 04:15:39.351603
SA113_GAIA_patch_Willeke 5.9286111e-05          2020-05-20 16:03:58.074205
2MASS-PSC 5.929121323117e-05          2020-07-15 10:48:18.016710
2MASS-PSC 5.92587053205e-05          2020-07-15 11:07:15.672871
GAIA DR2 5.925870532047e-05          2020-07-16 11:45:18.708324


<a id = "REF3.1" REF></a>
### Creating new SLs with the new astrometrical solutions

Producing SourceLists with Gaia as REFCAT



In [7]:
for slid in SLIDlist:
    redsframe = (SourceList.SLID == slid)[0].frame
    redsfile = (SourceList.SLID == slid)[0].frame.filename
    creationdate = (AstrometricParameters.reduced == redsframe).order_by('creation_date')[-1]

    query = (ReducedScienceFrame.filename == redsfile)
    frame = query[0]
    astrom = (AstrometricParameters.reduced == frame).max('creation_date')
    print(astrom.process_params.REFCAT)
    print('-----------------------')
    frame.retrieve()
    frame.weight.retrieve()
    sourcelist = SourceList()
    sourcelist.frame = frame
    sourcelist.sexparam = ['MAG_APER', 'MAGERR_APER', 'FLUX_AUTO', 'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ERRX2_IMAGE', 'ERRY2_IMAGE', 'ERRXY_IMAGE', 'XWIN', 'YWIN', 'ERRX2WIN', 'ERRY2WIN', 'ERRXYWIN', 'AWIN', 'BWIN', 'ERRAWIN', 'ERRBWIN', 'ERRTHETAWIN', 'THETAWIN', 'X2WIN', 'Y2WIN', 'XYWIN', 'CXXWIN', 'CYYWIN', 'CXYWIN', 'ERRCXXWIN', 'ERRCYYWIN', 'ERRCXYWIN']
    sourcelist.astrom_params = astrom
    sourcelist.make()
    sourcelist.commit()


GAIA-DR2
-----------------------
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:05 - Retrieving Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4509118-d4b707aaab715eea0e4f31d755a17cae52ea6fe6.fits
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:08 - Retrieved Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4509118-d4b707aaab715eea0e4f31d755a17cae52ea6fe6.fits[32807kB] in 3.46 seconds (9472.47kBps)
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:08 - Retrieving Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Wei-58969.4510008-fbcbd5399c7f92c57b7e7e0286dfb6da63e64373.fits.gz
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:11 - Retrieved Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Wei-58969.4510008-fbcbd5399c7f92c57b7e7e0286dfb6da63e64373.fits.gz[24192kB] in 2.12 seconds (11418.36kBps)
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:11 - Not keeping Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Wei-58969.4510008-fbcbd5399c7f92c57b7e7e0286dfb6da63

[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:43 - RUNNING: ldacconv -i tmp1595317481.641494.002a4571-50e1-4123-b020-874fd07407b7.sex.cat -o SL-catalog.tmp1595317481.641494.002a4571-50e1-4123-b020-874fd07407b7.cat -b 1 -c OMEGACAM -f "SloanR"
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:43 - RUNNING: ldacrenkey -i SL-catalog.tmp1595317481.641494.002a4571-50e1-4123-b020-874fd07407b7.cat -o renkey.dat -t OBJECTS -k Theta POSANG
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:43 - RUNNING: ldacrenkey -i SL-catalog.tmp1595317481.641494.002a4571-50e1-4123-b020-874fd07407b7.cat -o renkey.dat -t OBJECTS -k ALPHA_SKY RA
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:43 - RUNNING: ldacrenkey -i SL-catalog.tmp1595317481.641494.002a4571-50e1-4123-b020-874fd07407b7.cat -o renkey.dat -t OBJECTS -k DELTA_SKY DEC
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:44 - Outer bounding box: (325.22060,  0.76060,325.10012,  0.76037,325.09976,  1.00181,325.22027,  1.00204 )
[coma-2.vm.astro.rug.nl] 2020-07-21T09:44:46 - Created Sou

[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:10 - Retrieved Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Wei-58969.5188856-2e5b3f99a9666a9d8a3d58f8096ed95a219c52ad.fits.gz[24575kB] in 1.07 seconds (22898.11kBps)
[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:10 - Not keeping Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Wei-58969.5188856-2e5b3f99a9666a9d8a3d58f8096ed95a219c52ad.fits.gz
[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:10 - Transferred Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Wei-58969.5188856-2e5b3f99a9666a9d8a3d58f8096ed95a219c52ad.fits.gz[24575kB] -> Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Wei-58969.5188856-2e5b3f99a9666a9d8a3d58f8096ed95a219c52ad.fits[32805kB] (0.55sec)
[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:10 - Saving tmp1595317510.579194.bb1712d8-0cab-40d1-a08e-e41733e56153-realframe-Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.5188061-991426e55bfc3d88be7f892404cf5297861531fc.fits
[coma-2.vm.astro.r

[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:39 - Outer bounding box: (325.22004,  0.75517,325.09919,  0.75495,325.09871,  0.99720,325.21960,  0.99741 )
[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:42 - Created SourceList with SLID = 76556881, name = SL-WMULDER-0076556881 and 310 sources
GAIA-DR2
-----------------------
[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:42 - Retrieving Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.3899330-d1b3ab02d8739df33f6f8d97ca20a06958ae970b.fits
[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:45 - Retrieved Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.3899330-d1b3ab02d8739df33f6f8d97ca20a06958ae970b.fits[32807kB] in 2.73 seconds (12020.17kBps)
[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:45 - Retrieving Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Wei-58969.3900067-38f1b03cc4c358acaf52dbef4ede08c019ddb2a6.fits.gz
[coma-2.vm.astro.rug.nl] 2020-07-21T09:45:47 - Retrieved Cal-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#

[coma-2.vm.astro.rug.nl] 2020-07-21T09:46:33 - Using SExtractor version 2.8.6
[coma-2.vm.astro.rug.nl] 2020-07-21T09:46:33 - Running: sex tmp1595317577.504084.bb4f78b3-8551-4ffd-b6e2-d1ab82d79be2-realframe-Sci-WMULDER-OMEGACAM-------OCAM_r_SDSS-ESO_CCD_#90-Red---Sci-58969.4806177-34c3c97e5d35c82f6ce0262c776e1d5c4ef864f5.fits -c tmp1595317577.503387.af12fc66-875c-4683-a32f-436d3ae26300.SExtractor.conf
[coma-2.vm.astro.rug.nl] 2020-07-21T09:46:35 - RUNNING: ldacconv -i tmp1595317577.504084.bb4f78b3-8551-4ffd-b6e2-d1ab82d79be2.sex.cat -o SL-catalog.tmp1595317577.504084.bb4f78b3-8551-4ffd-b6e2-d1ab82d79be2.cat -b 1 -c OMEGACAM -f "SloanR"
[coma-2.vm.astro.rug.nl] 2020-07-21T09:46:35 - RUNNING: ldacrenkey -i SL-catalog.tmp1595317577.504084.bb4f78b3-8551-4ffd-b6e2-d1ab82d79be2.cat -o renkey.dat -t OBJECTS -k Theta POSANG
[coma-2.vm.astro.rug.nl] 2020-07-21T09:46:35 - RUNNING: ldacrenkey -i SL-catalog.tmp1595317577.504084.bb4f78b3-8551-4ffd-b6e2-d1ab82d79be2.cat -o renkey.dat -t OBJECTS -k AL

In [9]:
SLIDlist_gaia_ref = [76556811, 76556821, 76556831, 76556841, 76556851, 76556861, 76556871, 76556881, 76556891, 76556901, 76556911, 76556921]

In [11]:
novicelists = [(SourceList.SLID == slid)[0] for slid in SLIDlist_gaia_ref]
print(novicelists)

[<astro.main.SourceList.SourceList object at 0x7f02bb3eb1d0>, <astro.main.SourceList.SourceList object at 0x7f02bb3eb860>, <astro.main.SourceList.SourceList object at 0x7f02ba4dfbe0>, <astro.main.SourceList.SourceList object at 0x7f02ba4888d0>, <astro.main.SourceList.SourceList object at 0x7f02ba4885c0>, <astro.main.SourceList.SourceList object at 0x7f02ba1139b0>, <astro.main.SourceList.SourceList object at 0x7f02bb39f160>, <astro.main.SourceList.SourceList object at 0x7f02bb39fd30>, <astro.main.SourceList.SourceList object at 0x7f02bb39f7f0>, <astro.main.SourceList.SourceList object at 0x7f02ba54ed68>, <astro.main.SourceList.SourceList object at 0x7f02b98e17f0>, <astro.main.SourceList.SourceList object at 0x7f02b98e14a8>]


<a id = "REF3.2" REF></a>
### Creating new AL with the new SLs

In [12]:
run = 'no'

SL_GAIAmastercatalog = (SourceList.SLID == 76551301)[0]

masterlist = SL_GAIAmastercatalog
novicelists = [(SourceList.SLID == slid)[0] for slid in SLIDlist_gaia_ref]
nr_sourcelists = len(novicelists)

if run == 'yes':
    import datetime
    today = datetime.date.today().strftime("%y%m%d")
    AL0_name = 'MasterTableGAIAMASTERCat{}_{}_0'.format(nr_sourcelists, today)
    print("Name AssociateList: {}".format(AL0_name))

    sourcelists=novicelists

    AL0 = AssociateList()
    AL0.input_lists.append(masterlist)
    AL0.input_lists.append(novicelists[0])
    AL0.set_search_distance(5.0)
    AL0.associatelisttype = 2
    AL0.process_params.SEXTRACTOR_FLAG_MASK = 255
    AL0.name = AL0_name
    AL0.make()
    AL0.commit()

    AL_previous = AL0
    i=0
    for sl in novicelists[1:]:
        i = i+1
        AL_name = 'MasterTableGAIAMASTERCat{}_{}_{}'.format(nr_sourcelists, today, i)
        print("Creating: {}".format(AL_name))

        AL=AssociateList()
        AL.input_lists.append(AL_previous)
        AL.input_lists.append(sl)
        AL.set_search_distance(5.0)
        AL.associatelisttype = 2
        AL.process_params.SEXTRACTOR_FLAG_MASK = 255
        AL.name = AL_name
        AL.make()
        AL.commit()
        AL_previous = AL




Name AssociateList: MasterTableGAIAMASTERCat12_200721_0
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Found 309 sources in SourceList with SLID = 76556811
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Filtered out 11 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Found 220 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:10 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Creat

[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:27 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:27 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:27 - Created Master AssociateList with ALID = 2708211, name = "MasterTableGAIAMASTERCat12_200721_7" and 315 associates!
Creating: MasterTableGAIAMASTERCat12_200721_8
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Found 295 sources in SourceList with SLID = 76556891
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Filtered out 10 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Found 220 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Insertin

NameError: name 'novicelist' is not defined

``` bash
Name AssociateList: MasterTableGAIAMASTERCat12_200721_0
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Found 309 sources in SourceList with SLID = 76556811
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Filtered out 11 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Found 220 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:09 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:10 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Created Master AssociateList with ALID = 2708141, name = "MasterTableGAIAMASTERCat12_200721_0" and 319 associates!
Creating: MasterTableGAIAMASTERCat12_200721_1
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Found 280 sources in SourceList with SLID = 76556821
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:11 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:12 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:12 - Filtered out 9 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:12 - Found 215 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:12 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:12 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:12 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:12 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Created Master AssociateList with ALID = 2708151, name = "MasterTableGAIAMASTERCat12_200721_1" and 295 associates!
Creating: MasterTableGAIAMASTERCat12_200721_2
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Found 213 sources in SourceList with SLID = 76556831
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Filtered out 5 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Found 187 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:20 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:21 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:21 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:21 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:21 - Created Master AssociateList with ALID = 2708161, name = "MasterTableGAIAMASTERCat12_200721_2" and 256 associates!
Creating: MasterTableGAIAMASTERCat12_200721_3
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:21 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:21 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:21 - Found 306 sources in SourceList with SLID = 76556841
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:21 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:22 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:22 - Filtered out 11 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:22 - Found 218 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:22 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:22 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:22 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:22 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Created Master AssociateList with ALID = 2708171, name = "MasterTableGAIAMASTERCat12_200721_3" and 318 associates!
Creating: MasterTableGAIAMASTERCat12_200721_4
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Found 181 sources in SourceList with SLID = 76556851
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Filtered out 6 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Found 168 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:23 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Created Master AssociateList with ALID = 2708181, name = "MasterTableGAIAMASTERCat12_200721_4" and 243 associates!
Creating: MasterTableGAIAMASTERCat12_200721_5
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Found 286 sources in SourceList with SLID = 76556861
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Filtered out 9 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Found 220 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:24 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Created Master AssociateList with ALID = 2708191, name = "MasterTableGAIAMASTERCat12_200721_5" and 296 associates!
Creating: MasterTableGAIAMASTERCat12_200721_6
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Found 295 sources in SourceList with SLID = 76556871
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Filtered out 9 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Found 221 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:25 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Created Master AssociateList with ALID = 2708201, name = "MasterTableGAIAMASTERCat12_200721_6" and 304 associates!
Creating: MasterTableGAIAMASTERCat12_200721_7
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Found 307 sources in SourceList with SLID = 76556881
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Filtered out 10 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Found 222 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:26 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:27 - Inserting second half of pairs

[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:27 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:27 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:27 - Created Master AssociateList with ALID = 2708211, name = "MasterTableGAIAMASTERCat12_200721_7" and 315 associates!
Creating: MasterTableGAIAMASTERCat12_200721_8
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Found 295 sources in SourceList with SLID = 76556891
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Filtered out 10 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Found 220 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:28 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Created Master AssociateList with ALID = 2708221, name = "MasterTableGAIAMASTERCat12_200721_8" and 305 associates!
Creating: MasterTableGAIAMASTERCat12_200721_9
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Found 275 sources in SourceList with SLID = 76556901
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Filtered out 9 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Found 218 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:29 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:30 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:30 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:30 - Created Master AssociateList with ALID = 2708231, name = "MasterTableGAIAMASTERCat12_200721_9" and 287 associates!
Creating: MasterTableGAIAMASTERCat12_200721_10
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:30 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:30 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:30 - Found 300 sources in SourceList with SLID = 76556911
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:30 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:31 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:31 - Filtered out 9 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:31 - Found 215 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:31 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:31 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:31 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:31 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Created Master AssociateList with ALID = 2708241, name = "MasterTableGAIAMASTERCat12_200721_10" and 315 associates!
Creating: MasterTableGAIAMASTERCat12_200721_11
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Preparing for the matching
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Found 230 sources in SourceList with SLID = 76551301
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Found 304 sources in SourceList with SLID = 76556921
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Looking for pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Looking for closest pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Filtered out 8 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Found 214 pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Inserting first half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Inserting second half of pairs
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Inserting null associations from last input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:32 - Inserting null associations from first input list
[coma-2.vm.astro.rug.nl] 2020-07-21T12:44:33 - Created Master AssociateList with ALID = 2708251, name = "MasterTableGAIAMASTERCat12_200721_11" and 320 associates!

```

In [14]:
AL = (AssociateList.ALID == 2708251)[0].info()

AssociateList: Name of AssociateList : MasterTableGAIAMASTERCat12_200721_11
AssociateList ID      : 2708251
Associates in list    : 320

 |
 +-ALID: 2708251
 +-OBJECT: None;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT;STD,ZEROPOINT
 +-associatelisttype: 2
 +-associates: {}
 +-creation_date: 2020-07-21 10:44:33.347925
 +-filename: 
 +-globalname: 
 +-inputassociatelist: Name of AssociateList : MasterTableGAIAMASTERCat12_200721_10
AssociateList ID      : 2708241
Associates in list    : 315

 +-is_valid: 1
 +-llDEC: 0.7613800459711609
 +-llRA: 325.2190480267468
 +-lrDEC: 0.7613800902413959
 +-lrRA: 325.102010565871
 +-name: MasterTableGAIAMASTERCat12_200721_11
 +-number_of_associates: 320
 +-object_id: 'AAA5204A603C52F3E053174A17AC5E3F'
 +-process_params: <astro.main.AssociateList.AssociateListParameters object at 0x7f02baffa4a8>
 +-sourcelists: <class 'common.database.typed_list.l